# Projet télédétection
Audrey Zimmer

## 1. Import des librairies

In [12]:
# Librairies python
import sys
sys.path.append('..')
import os
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report, \
    accuracy_score
import geopandas as gpd
from osgeo import ogr
from osgeo import gdal
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedStratifiedKFold, KFold


# Librairies personnelles
from libsigma import classification as cla
from libsigma import read_and_write as rw
from libsigma import plots


## Création des dossiers

In [13]:
my_folder = "/home/onyxia/work/Projet_teledetection"
results_path = os.path.join(my_folder, "../results")
figure_path = os.path.join(my_folder, "../results/figure")
img_path = os.path.join(my_folder, "img")

## Formatage des données

In [ ]:
b03 = '/home/onyxia/work/data/projet_eval/bretagne_23-24_B03.tif'
b03_data_set = gdal.Open(b03, gdal.GA_Update)
type(data_set)

b05 = '/home/onyxia/work/data/projet_eval/bretagne_23-24_B05.tif'
b05_data_set = gdal.Open(b05, gdal.GA_Update)
type(data_set)

osgeo.gdal.Dataset

In [16]:
print('Nombre de colonnes b03 :', data_set.RasterXSize)
print('Nombre de lignes b03 :', data_set.RasterYSize)
print('Nombre de bandes b03 :', data_set.RasterCount)

print('Nombre de colonnes b05 :', data_set.RasterXSize)
print('Nombre de lignes b05 :', data_set.RasterYSize)
print('Nombre de bandes b05 :', data_set.RasterCount)

Nombre de colonnes b03 : 1533
Nombre de lignes b03 : 612
Nombre de bandes b03 : 5
Nombre de colonnes b05 : 1533
Nombre de lignes b05 : 612
Nombre de bandes b05 : 5


Les bandes 03 et 05 qui sont utilisées pour la suite de l'étude contiennent le même nombre de colonnes (1533), le même nombre de lignes (612) et le même nombre de bandes (5).

## Analyse des échantillons
### Nombre d'échantillons